# Incompressible s/pimpleFoam Models

## Section 1: Introduction 

## Section 2: Information from Tutorials 

## Section 3: Setting of Turbulent Quantities 

### Set independent values

In [1]:
turb_intensity = 0.05 

0.05

In [2]:
outerDiameter = 3.0; innerDiamater = 0.9; hydraulicDiameter = outerDiameter - innerDiamater;

In [3]:
# set interField(U)
internalField_U = 5 

5

### Set dependent values

In [4]:
# compute internalField(k)
internalField_k = 1.5 * (internalField_U * 0.05)^2

0.09375

In [5]:
# compute turbulent length scale for inlet-air and for computational domain overall 
inlet_air_turb_len_scale = 0.038 * 4.2

0.1596

In [6]:
# compute turbulent length scale for inlet-fuel 
inlet_fuel_turb_len_scale = 0.038 * 0.16 

0.00608

In [7]:
# compute internalField(epsilon) = C_{\mu}^{0.75} k^(3/2) 1/turbulent-length-scale
internalField_epsilon = (0.09)^(0.75)*(internalField_k)^(3/2) / 0.1596 

0.02955329505036729

In [8]:
# compute internalField(nut) = C_{\mu} * internalField(k)^2 / internalField(epsilon)
internalField_nut = 0.09 * internalField_k^2 / internalField_epsilon 

0.026765733690672477

## Section 4: simpleFoam Runs 

### 0-folder 

##### General Considerations 

1. It is important to set initial values such that k, epsilon and nut are physically consistent. Information on how to set these initial values is provided e.g. [here](https://www.openfoam.com/documentation/tutorial-guide/3-compressible-flow/3.1-steady-turbulent-flow-over-a-backward-facing-step);
2. In the discussion below we assume that the internalField value for velocity has been set equal to internalField(v).   

#### p-file 

#### U-file 

1. inlet_air: (0, inlet_air(v), 0) where for time being inlet_air(v) = 10 m/s;

2. inlet_fuel: (0, inlet_fuel(v), 0) where for time being inlet_fuel(v) = 0;

3. internalField(U): (0, internalField(v), 0) where internalField(v) = 0.5 * inlet_air(v) = 5 m/s;

4. Two questions: Q1: assume that inlet_fuel(v) = 0 (and thus that only inlet_air(v) <> 0). Is it more beneficial to set internalField(v) = 0 or to set internalField(v) = inlet_air(v). We currently imagine that internalField(v) = 0 is required (best?) to air jet evolve into the computational domain. Q2: assume that inlet_fuel(v) <> 0. Does setting for internalField(v) require to be modified? We currently imagine that this is not required and that internalField(v) = 0 remains the best option. 


#### k-file 
1. Notes
    1. Should we use [type turbulentIntensityKinecticEnergy](https://www.openfoam.com/documentation/guides/latest/doc/guide-bcs-inlet-turbulent-k-turbulent-intensity-kinetic-energy.html) or type fixedValue? 
    2. Fomer case: turbulentIntensityKinecticEnergy inherits from InletOutletPatchScalarField and is able to impose a zero-gradient condition in case of reverse flow. The entry value is merely a placeholder. Its value should therefore not affect simulations?  
    3. Latter case: how to ensure that inlet_air(k) changes each time inlet_air(v) is changed in parametric studies or in specifying a mass flow rate for the velocity instead? 

2. inlet_air: use turbulentIntensityKinecticEnergy with turbulent intensity 0.05 percent

3. inlet_fuel: use turbulentIntensityKinecticEnergy with intensity possibly different from above

4. internalField(k): Assume that the turbulent intensity value turb-intensity has been set to a value between 1 and 5 percent. We assume that the turbulent intensity for the inlet_air patch (largest in surface area) applies to the entire (volumetric) domain. Then set 
    internalField(k) = 1.5 * ( internalField(v) * inlet_air_turb-intensity )^2 = 1.5 * (5 * 0.05)^2 = 0.09375 (in units m^2 / s^2 )

#### epsilon-file 

1. Notes 
    1. As before: Should we use [type turbulentMixingLengthDissipationRateInlet](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1turbulentMixingLengthDissipationRateInletFvPatchScalarField.html) or type fixedValue? 
    2. Former case: turbulentMixingLengthDissipationRateInlet inherits from InletOutletPatchScalarField and is able to impose a zero-gradient condition in case of reverse flow. The entry value is merely a placeholder. Its value should therefore not affect simulations?
    3. Latter case: how to ensure that inlet_air(epsilon) changes each time inlet_air(v) is changed in parametric studies or in specifying a mass flow rate for the velocity instead? 
    4. We will use the hydraulic diameter of an annulus to compute the turbulent length scale in the domain. The [hydraulic diamater](https://www.ihsenergy.ca/support/documentation_ca/Harmony/content/html_files/reference_material/calculations_and_correlations/annular_diameters.htm) hydraulic-D is defined as hydraulic-D = D_2 - D_1 where D_2 and D_1 are the outer and inner radius of the annulus. D_2 = 2 * 3 = 6 meter. D_1 = 2 * 0.9 = 1.8 meter. Thus hydraulic-D = 4.2 meter. 
    5. We will use as turbulent length scale L_T = 0.038 * hydraulic-D = 0.038 * 4.2 meter = 0.1596 meter. 
    6. The fuel pipe has radius 0.08 meter and thus diameter 0.16 meter. The turbulent length scale of the fuel pipe has thus a turbulent length scale equal to inlet_fuel_L_T = 0.038 * 0.16 = 0.00608 

2. inlet_air: use turbulentMixingLengthDissipationRateInlet with turbulent scale equal to 3.8 percent of the hydraulic diameter of inlet_air; 

3. inlet_fuel: use turbulentMixingLengthDissipationRateInlet with turbulent scale equal to 3.8 percent of the hydraulic diameter of inlet_fuel; 

4. internalField(epsilon): The turbulent dissipation rate epsilon  can be computed as epsilon = C_{\mu}^{3/4} k^{3/2} / turb-length-scale, where C_{\mu} = 0.09 and turb-length-scale is 3.8 percent of the hydraulic diameter. We assume that the turbulent length scale of the air patch applies to the entire domain. Then set internalField(epsilon) = (0.09)^{3/4} * internalField(k)^{3/2} / turb-length-scale = 0.02955 (in units 1/s); 

5. why is internalField set to 6500 (this value if large compare to our computations)? 

6. why is value on the patches set to internalField? 

#### nut-file 

1. Notes 
    1. Not sure how the [type calculated](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1calculatedFvPatchField.html) is used to enforce the boundary conditions. 

2. The turbulent kinematic viscosity nut  can be computed as nut = C_{\mu} k^2 / epsilon, where C_{\mu} = 0.09. Thus internalField(nut) = 0.09 * (internalField(k))^2 / internalField(epsilon) = 0.026766 (in units 1/s); 

3. on non-wall patches: set calculated 

4. on wall patches: set nutLowReWallFunction with standard settings 

### Numerical Results for Air-Fuel 

<div>
<img src="../figures/simpleFoam_airFuel_Uy.png" width=500/>
<center> Figure: simpleFoam axial velocity.  
    </center>
</div>

<div>
<img src="../figures/simpleFoam_airFuel_nut.png" width=500/>
<center> Figure: simpleFoam turbulent viscosity.  
    </center>
</div>

<div>
<img src="../figures/simpleFoam_rescon_airFuel.png" width=500/>
<center> Figure: simpleFoam convergence for air and fuel.  
    </center>
</div>

### Numerical Results for All Inlets  

<div>
<img src="../figures/simpleFoam_allInlets_Uy.png" width=500/>
<center> Figure: simpleFoam axial velocity.  
    </center>
</div>

<div>
<img src="../figures/simpleFoam_allInlets_nut.png" width=500/>
<center> Figure: simpleFoam turbulent viscosity.  
    </center>
</div>

<div>
<img src="../figures/simpleFoam_rescon _allInlets.png" width=500/>
<center> Figure: simpleFoam convergence for air, fuel and other inlets.  
    </center>
</div>

## Section 5: pimpleFoam Runs 